In [108]:
import requests
import urllib.request
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import re
import collections
import os

In [109]:
# Function to crawl for relevant URLs
def crawl_for_urls(starting_urls, max_urls=40):
    visited_urls = set()
    relevant_urls = collections.deque(starting_urls)
    while relevant_urls and len(visited_urls) < max_urls:
        url = relevant_urls.popleft()
        if url not in visited_urls:
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    visited_urls.add(url)
                    soup = BeautifulSoup(response.text, 'html.parser')
                    eachPageCounter = 0
                    for link in soup.find_all('a', href=True):
                        if eachPageCounter > 10:
                            break
                        # if 'spice' in link or 'Spice' in link or 'indian' in link or 'Indian' in link:
                        if 'hindi' in link or 'Hindi' in link or 'bollywoood' in link or 'Bollywood' in link:
                            abs_url = urljoin(url, link['href'])
                            if abs_url.startswith("http") and abs_url not in visited_urls and abs_url not in relevant_urls:
                                relevant_urls.append(abs_url)
                                eachPageCounter += 1
            except Exception as e:
                print(f"Error crawling {url}: {e}")
    return visited_urls

In [110]:
# Starting URLs for crawling
starting_urls = [
    # 'https://en.wikipedia.org/wiki/Indian_cuisine',
    # 'https://www.masterindian.com/blogs/master-indian-spice-blog/24-top-indian-spices-and-how-to-use-them',
    # 'https://en.wikipedia.org/wiki/List_of_Indian_spices',
    # 'https://badshahmasala.com/blogs/indian-spices-list/',
    # 'https://www.wellcurve.in/blog/types-of-spices/'
    'https://en.wikipedia.org/wiki/Hindi_cinema',
    'https://www.timeout.com/film/the-100-best-bollywood-movies-the-list',
    'https://www.bollycurves.com/resources/what-is-bollywood'
]

In [111]:
# Crawl for relevant URLs
relevant_urls = crawl_for_urls(starting_urls)

In [106]:
# Function to scrape text from each page
def scrape_text(urls):
    text_data = collections.defaultdict(str)
    for i, url in enumerate(urls):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                # text_data[url] = soup.get_text()
                # text_data[url] = soup.findAll(text=True)
                paras = soup.select('p')
                text = ''
                for p in paras:
                    text += p.get_text() + '\n'

                # Write text to a text file
                filename = f"page{i+1}.txt"
                filepath = os.path.join(filename)
                with open(filepath, 'w', encoding='utf-8') as f:
                    f.write(text)
                    text_data[url] = filepath
                    
        except Exception as e:
            print(f"Error scraping {url}: {e}")
    return text_data

In [107]:
# Scrape text from each page
text_data = scrape_text(relevant_urls)